In [1]:
import h5py
from tqdm import tqdm
import sqlite3
#import sqlite_utils as su

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('~/msmsgym/MSNovelist-private')
import os

import fp_management.database as db
import itertools
import uuid
import fp_management.mist_fingerprinting as fpr
import fp_management.fingerprint_map as fpm
import smiles_config as sc
import pickle

from rdkit import Chem
from rdkit.Chem import AllChem

In [2]:
db_crossval = "/home/stravsmi/msmsgym/msnovelist-data-0531/msnovelist_crossval.hdf5"
db_train = "/home/stravsmi/msmsgym/msnovelist-data-0531/msnovelist_train.hdf5"

# db_old = "/sirius6_db/combined_0824_v44.db"
db_uuid = uuid.uuid4()
db_new = f"/home/stravsmi/msmsgym/msnovelist-data-0531/mist-{db_uuid}.db"

h5_crossval = h5py.File(db_crossval, mode='r')
h5_train = h5py.File(db_train, mode='r')

PROCESSING_BLOCK_SIZE=40000
PROCESSING_BLOCK_MAX_COUNT=9999999999

In [3]:
%autoreload
fpr.MistFingerprinter.init_instance()
fingerprinter = fpr.MistFingerprinter.get_instance()

In [4]:
def try_fp_item(smiles_generic, smiles_canonical, fp):
    try:
        item = db.FpItem.fromFingerprint(
            smiles_generic = smiles_generic,
            smiles_canonical = smiles_canonical,
            fp = fp,
            source = "dataset",
            grp = "train",
            b64 = False
        )
    except:
        item = None
    return item

In [5]:

def db_item_block(block):
    smiles = [x for x in block if x  != b'\\N']
    smiles_proc = fingerprinter.process(smiles, calc_fingerprint=True, return_b64=False)
    fp = [x["fingerprint"] for x in smiles_proc]
    item = zip(smiles_proc, fp)
    fp_items_ = [try_fp_item(s['smiles_generic'], s['smiles_canonical'], fp)
                 for s, fp in item ]
    fp_items = [x for x in fp_items_ if x is not None]
    return fp_items

In [6]:
data_in = h5_train["smiles"]
data_iter = iter(data_in)

def take(n, iterable): 
    return list(itertools.islice(iterable, n))

print(f"database: {db_new}")

fp_db = db.FpDatabase.load_from_config(db_new)
block = take(PROCESSING_BLOCK_SIZE, data_iter)

KeyboardInterrupt: 

In [22]:
%autoreload
processed_blocks = 0
while (len(block) > 0) and (processed_blocks < PROCESSING_BLOCK_MAX_COUNT):
    print(f"Processing block {processed_blocks}")
    selected_elements = len(block)
    data_proc = db_item_block(block)
    print(f"Loaded {len(block)} elements, "
          f"selected {selected_elements} elements, "
          f"successfully processed {len(data_proc)} elements")
    fp_db.insert_fp_multiple(data_proc)
    #print(f"last inserted id: {inserted_id}")
    block = take(PROCESSING_BLOCK_SIZE, data_iter)
    processed_blocks = processed_blocks + 1
print("Done.")

Processing block 0
failed parsing id 180 - b'\\N'


RDKit ERROR: [15:38:24] SMILES Parse Error: syntax error while parsing: \N
RDKit ERROR: [15:38:24] SMILES Parse Error: Failed parsing SMILES '\N' for input: '\N'


Loaded 1000 elements, selected 1000 elements, successfully processed 999 elements
Done.


In [27]:
h5_train["smiles"][3179:3812]

array([b'CC1=CC=C(C=C1)S(=O)(=O)N2C(=NC(=N2)C3=CN=CC=C3)N', b'\\N',
       b'CC1=CC=CC=C1C2CC3=CC=CC=C3N=C(N2C)C',
       b'CN(C)P(=O)(C(CN1C=CN=C1)(O)P(=O)(OC)OC)OC',
       b'C1CCC(CC1)CCC(=O)OCC(=O)NC2=CC=C(C=C2)S(=O)(=O)N3CCOCC3',
       b'C1=CC(=CC=C1CC(C(=O)O)N(CCN(CC(=O)O)CC(=O)O)CC(=O)O)N=C=S',
       b'CCN(CC)CC(C)NC1=C2C=CC(=CC2=NC=C1)C(F)(F)F',
       b'CC1=C2C(=C(SC2=NC=C1)C(=O)NC3=C(N=CC=C3)C)N',
       b'CC(CCC1C(O1)(C)C(=O)OC)C2CCC3(C2(CC=C4C3CCC5C4(CCC(C5(C)C)O)C)C)C',
       b'CC(C)(C)NC(=O)C1CCCN1C(=O)C(C(CC2=CC=CC=C2)NC(=O)C=CC(=O)N)O',
       b'CN(CCC1CN(CCC1CC(=O)O)C(=O)C2=CC=CS2)C3=CC=CC=C3',
       b'B1(OC2CC3CC(C3(C)C)C2(O1)C)C(CCCCN)NC(=O)CN4C(=NC(=N4)CNC=O)CC5=CC=CC=C5',
       b'C1=CC=C2C(=C1)NC(=N2)C3=NC4=C(C=CC5=C4N=CC=C5)C=C3',
       b'CCC1C(C=CC=CC(=O)C(CC(C(C(C=CC(=O)O1)C)OC2C(C(CC(O2)C)N(C)C)O)CO)C)COC3C(C(C(C(O3)C)O)OC)OC',
       b'CCCCCCC=CCCCCCCCCC=COC(COC(=O)CCCCCCCC=CCC(=CCCCCC)C)COP(=O)(O)OCC[N+](C)(C)C',
       b'C1=CC=C2C(=C1)C=C3C4=C(C=CC=C24